In [146]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util
import pandas as pd

These commands need to run in the command line:

In [132]:
'''
export SPOTIPY_CLIENT_ID ='ac8bc9378a6c4029ade90be48171c12d'
export SPOTIPY_CLIENT_SECRET ='4a207f7d9e3943fc8496fb03c4f02ab0'
export SPOTIPY_REDIRECT_URI='http://localhost:8888'
'''

"\nexport SPOTIPY_CLIENT_ID ='ac8bc9378a6c4029ade90be48171c12d'\nexport SPOTIPY_CLIENT_SECRET ='4a207f7d9e3943fc8496fb03c4f02ab0'\nexport SPOTIPY_REDIRECT_URI='http://localhost:8888'\n"

In [133]:
USERNAME = 'sitararachel' #your spotify username
CLIENT_ID = 'ac8bc9378a6c4029ade90be48171c12d'#set at your developer account
CLIENT_SECRET = '4a207f7d9e3943fc8496fb03c4f02ab0' #set at your developer account
REDIRECT_URI = 'http://localhost:8888' #set at your developer account, usually "http://localhost:8000"
SCOPE = 'playlist-modify-public' # or else

In [134]:
token = util.prompt_for_user_token(username = USERNAME, 
                                   scope = SCOPE, 
                                   client_id = CLIENT_ID, 
                                   client_secret = CLIENT_SECRET, 
                                   redirect_uri = REDIRECT_URI)

if token:
   sp = spotipy.Spotify(auth=token)
   # dig your api


## 2. Use a playlist with songs I already like

In [99]:
source_user = 'yojabbanabba'
source_playlist_id = '3g2vJU3kCwK9P6cZRxxdhp'

In [100]:
sourcePlaylist = sp.user_playlist(source_user, source_playlist_id)

In [76]:
sourcePlaylist.keys()

dict_keys(['collaborative', 'description', 'external_urls', 'followers', 'href', 'id', 'images', 'name', 'owner', 'primary_color', 'public', 'snapshot_id', 'tracks', 'type', 'uri'])

In [31]:
sourcePlaylist['tracks'].keys()

dict_keys(['href', 'items', 'limit', 'next', 'offset', 'previous', 'total'])

In [44]:
sourcePlaylist['tracks']['items'][3].keys()

dict_keys(['added_at', 'added_by', 'is_local', 'primary_color', 'track', 'video_thumbnail'])

In [45]:
sourcePlaylist['tracks']['items'][3]['track'].keys()

dict_keys(['album', 'artists', 'available_markets', 'disc_number', 'duration_ms', 'episode', 'explicit', 'external_ids', 'external_urls', 'href', 'id', 'is_local', 'name', 'popularity', 'preview_url', 'track', 'track_number', 'type', 'uri'])

In [101]:
tracks = sourcePlaylist["tracks"]
songs = tracks["items"] 

tracks is a dictionary of tracks metadata.

songs is a list of dictionaries with song metadata.

In [87]:
target_user = 'sitararachel'
target_playlist_good = '6aNd523vJgC0I8SLvN7wID'

In [88]:
while tracks['next']:
    tracks = sp.next(tracks)
    for item in tracks["items"]:
        songs.append(item)

In [89]:
ids = [] 
print(len(songs))
print(songs[0]['track']['id'])
i = 0
for i in range(len(songs)):
    sp.user_playlist_add_tracks(target_user, target_playlist_good, [songs[i]["track"]["id"]])

720
6iajHa34cSiD5s42Cq9miJ


In [104]:
playlists = [('sitararachel','6kRSiAQzP8MoI6ZAhSoUi0'),('Spotify','37i9dQZF1DX5kjCvsC5isB')]

In [112]:
for p in playlists:
    sourcePlaylist = sp.user_playlist(p[0], p[1])
    
    tracks = sourcePlaylist["tracks"]
    songs = tracks["items"] 
    
    while tracks['next']:
        tracks = sp.next(tracks)
        for item in tracks["items"]:
            songs.append(item)
    i = 0
    for i in range(len(songs)):
        sp.user_playlist_add_tracks(target_user, target_playlist_good, [songs[i]["track"]["id"]])

## 3. Playlist of "bad" songs:

In [124]:
playlists = [('Spotify','37i9dQZF1DX4dyzvuaRJ0n'),('Spotify','37i9dQZF1DX55dNU0PWnO5'),
             ('Spotify','37i9dQZF1DXbl9rMxGEmRC'),('Spotify','37i9dQZF1DX1OIMC8iDi74')]

In [125]:
target_playlist_bad = '1YochZHuIlxx1nXT44uDGX'

In [130]:
for p in playlists:
    sourcePlaylist = sp.user_playlist(p[0], p[1])
    
    tracks = sourcePlaylist["tracks"]
    songs = tracks["items"] 
    
    while tracks['next']:
        tracks = sp.next(tracks)
        for item in tracks["items"]:
            songs.append(item)
    i = 0
    for i in range(len(songs)):
        sp.user_playlist_add_tracks(target_user, target_playlist_bad, [songs[i]["track"]["id"]])

## 4. Get the song IDs for each song in the playlists

In [137]:
good_playlist = sp.user_playlist("sitararachel", "6aNd523vJgC0I8SLvN7wID")
good_tracks = good_playlist["tracks"]
good_songs = good_tracks["items"] 
while good_tracks['next']:
    good_tracks = sp.next(good_tracks)
    for item in good_tracks["items"]:
        good_songs.append(item)

In [138]:
good_ids = [] 
for i in range(len(good_songs)):
    good_ids.append(good_songs[i]['track']['id'])

In [140]:
bad_playlist = sp.user_playlist("sitararachel", "6aNd523vJgC0I8SLvN7wID")
bad_tracks = good_playlist["tracks"]
bad_songs = good_tracks["items"] 
while bad_tracks['next']:
    bad_tracks = sp.next(bad_tracks)
    for item in bad_tracks["items"]:
        bad_songs.append(item)

In [141]:
bad_ids = [] 
for i in range(len(bad_songs)):
    bad_ids.append(bad_songs[i]['track']['id'])

## 5. Pull track features into a dataframe

In [142]:
features = []
for i in range(0,len(good_ids),50):
    audio_features = sp.audio_features(good_ids[i:i+50])
    for track in audio_features:
        features.append(track)
        features[-1]['target'] = 1

In [143]:
for i in range(0,len(bad_ids),50):
    audio_features = sp.audio_features(bad_ids[i:i+50])
    for track in audio_features:
        features.append(track)
        features[-1]['target'] = 0

In [147]:
trainingData = pd.DataFrame(features)

In [148]:
trainingData

,acousticness,analysis_url,danceability,duration_ms,energy,id,instrumentalness,key,liveness,loudness,mode,speechiness,target,tempo,time_signature,track_href,type,uri,valence
0,0.008870,https://api.spotify.com/v1/audio-analysis/6iaj...,0.706,225947,0.537,6iajHa34cSiD5s42Cq9miJ,0.000000,7,0.1290,-6.347,1,0.3480,1,203.746,4,https://api.spotify.com/v1/tracks/6iajHa34cSiD...,audio_features,spotify:track:6iajHa34cSiD5s42Cq9miJ,0.7750
1,0.104000,https://api.spotify.com/v1/audio-analysis/4oiA...,0.652,249960,0.621,4oiAlcisfimXQc1wyAtX9a,0.000005,9,0.0382,-8.677,0,0.2630,1,169.983,4,https://api.spotify.com/v1/tracks/4oiAlcisfimX...,audio_features,spotify:track:4oiAlcisfimXQc1wyAtX9a,0.6610
2,0.603000,https://api.spotify.com/v1/audio-analysis/3XVB...,0.609,228707,0.445,3XVBdLihbNbxUwZosxcGuJ,0.000007,7,0.1040,-9.129,1,0.1060,1,118.393,3,https://api.spotify.com/v1/tracks/3XVBdLihbNbx...,audio_features,spotify:track:3XVBdLihbNbxUwZosxcGuJ,0.1660
3,0.001820,https://api.spotify.com/v1/audio-analysis/0GLU...,0.593,273053,0.680,0GLUBbX4daHJkT3RQHEOia,0.000048,7,0.3100,-5.556,1,0.3350,1,84.547,4,https://api.spotify.com/v1/tracks/0GLUBbX4daHJ...,audio_features,spotify:track:0GLUBbX4daHJkT3RQHEOia,0.4830
4,0.384000,https://api.spotify.com/v1/audio-analysis/1yeL...,0.783,229800,0.758,1yeLSqz8u8ip9NnzjGs7lQ,0.000000,10,0.0938,-2.663,0,0.1510,1,89.434,4,https://api.spotify.com/v1/tracks/1yeLSqz8u8ip...,audio_features,spotify:track:1yeLSqz8u8ip9NnzjGs7lQ,0.7800
5,0.197000,https://api.spotify.com/v1/audio-analysis/7rOI...,0.701,258667,0.601,7rOIWUhSfjWh4E7443PYE8,0.000000,11,0.0649,-7.678,0,0.1530,1,83.433,4,https://api.spotify.com/v1/tracks/7rOIWUhSfjWh...,audio_features,spotify:track:7rOIWUhSfjWh4E7443PYE8,0.5640
6,0.080400,https://api.spotify.com/v1/audio-analysis/09BK...,0.548,159787,0.764,09BKt4jvY0KZLwfHfP2EEy,0.000055,11,0.0774,-6.586,0,0.1040,1,90.027,4,https://api.spotify.com/v1/tracks/09BKt4jvY0KZ...,audio_features,spotify:track:09BKt4jvY0KZLwfHfP2EEy,0.6610
7,0.164000,https://api.spotify.com/v1/audio-analysis/4B9z...,0.831,251267,0.686,4B9zlmKyr1L2wYOoaSAA31,0.000000,7,0.4670,-9.796,1,0.1020,1,119.970,4,https://api.spotify.com/v1/tracks/4B9zlmKyr1L2...,audio_features,spotify:track:4B9zlmKyr1L2wYOoaSAA31,0.5830
8,0.265000,https://api.spotify.com/v1/audio-analysis/3kgu...,0.699,254467,0.668,3kgutGd839IOYRl6ekeq6V,0.000000,1,0.1410,-5.305,1,0.1060,1,83.074,4,https://api.spotify.com/v1/tracks/3kgutGd839IO...,audio_features,spotify:track:3kgutGd839IOYRl6ekeq6V,0.4720
9,0.037400,https://api.spotify.com/v1/audio-analysis/4HSA...,0.933,236480,0.701,4HSAJpNocVNJbwbQvtCMdO,0.000037,7,0.1440,-5.603,1,0.1730,1,112.965,4,https://api.spotify.com/v1/tracks/4HSAJpNocVNJ...,audio_features,spotify:track:4HSAJpNocVNJbwbQvtCMdO,0.6120


In [155]:
df = trainingData.to_csv(r'sitaraabraham\Documents\Springboard\Capstone #2\spotify.csv')